In [1]:
import pandas as pd
import numpy as np
train=pd.read_csv('~/SecretGarden/DeepLearning/RecSy/trivagodataset/train_drop_duplicates_and_after_last_clickout_items.csv')
train['impressions']=train['impressions'].str.split("|")
train.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
2,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
3,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
4,00RL8Z82B2Z1,aff3928535f48,1541037542,9,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN


In [30]:
train_item_based=train[train["reference"].str.isnumeric()]
click_out=train.dropna()

In [ ]:
session_lengths = click_out.groupby('session_id').size()
click_out = click_out[np.in1d(click_out.session_id, session_lengths[session_lengths>1].index)]
item_supports = data.groupby('ItemId').size()
data = data[np.in1d(data.ItemId, item_supports[item_supports>=5].index)]
session_lengths = data.groupby('SessionId').size()

In [24]:
def create_seq_db_filter_top_k(file, topk=0,session_gate=1):
#     c = Counter(list(file['reference']))

#     if topk > 1:
#         keeper = set([x[0] for x in c.most_common(topk)])
#         file = file[file['reference'].isin(keeper)]
        
    drop_session=pd.DataFrame(file.groupby('session_id').size())
    drop_session=drop_session[drop_session>1].dropna()
    click_out=pd.merge(file,drop_session,"inner",on=['session_id','session_id'])
    file=click_out.reset_index()

    # group by session id and concat song_id
    groups = file.groupby('session_id')

    # convert item ids to string, then aggregate them to lists
    aggregated = groups['reference'].agg({'sequence': lambda x: list(map(str, x))})
    init_ts = groups['timestamp'].min()
    users = groups['user_id'].min()  # it's just fast, min doesn't actually make sense

    result = aggregated.join(init_ts).join(users)
    result.reset_index(inplace=True)
    return result

In [31]:
from collections import Counter
data_util=create_seq_db_filter_top_k(click_out)
data_util.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version


,session_id,sequence,timestamp,user_id
0,000138ab4f789,"[2423456, 2423456, 152444]",1541446487,WDBP226XSN9E
1,0001c758d8d1f,"[3876726, 5970084]",1541347999,QF54BRJZ12IN
2,0002cff5d90c1,"[17434, 17434]",1541447614,3SQFCOFW881Z
3,0003da0ef2b0a,"[2618252, 2618252]",1541335935,3YCNEOPZNI69
4,00052d679e39b,"[846731, 1336638, 40734]",1541180413,B4GJAVIIU6AM


In [32]:
cnt = Counter()
data_util.sequence.map(cnt.update)
sequence_length = data_util.sequence.map(len).values
n_sessions_per_user = data_util.groupby('user_id').size()

print('Number of items: {}'.format(len(cnt)))
print('Number of users: {}'.format(data_util.user_id.nunique()))
print('Number of sessions: {}'.format(len(data_util)) )

Number of items: 103228
Number of users: 72161
Number of sessions: 78668


In [27]:
def last_session_out_split(data,
                           user_key='user_id',
                           session_key='session_id',
                           time_key='timestamp'):
    """
    Assign the last session of every user to the test set and the remaining ones to the training set
    """
    sessions = data.sort_values(by=[user_key, time_key]).groupby(user_key)[session_key]
    last_session = sessions.last()
    train = data[~data.session_id.isin(last_session.values)].copy()
    test = data[data.session_id.isin(last_session.values)].copy()
    return train, test

In [28]:
data.sort_values(by=[user_key, time_key]).groupby(user_key)[session_key]

NameError: name 'data' is not defined

In [29]:
train_data, test_data = last_session_out_split(data_util)
print("Train sessions: {} - Test sessions: {}".format(len(train_data), len(test_data)))

Train sessions: 6507 - Test sessions: 72161


In [22]:
def dataset_to_format(dataset):
    """
    Convert a list of sequences to GRU4Rec format.
    Based on this StackOverflow answer: https://stackoverflow.com/a/48532692

    :param dataset: the dataset to be transformed
    """

    lst_col = 'sequence'
    df = dataset.reset_index()
    unstacked = pd.DataFrame({
        col: np.repeat(df[col].values, df[lst_col].str.len()) for col in df.columns.drop(lst_col)}
    ).assign(**{lst_col: np.concatenate(df[lst_col].values)})[df.columns]
    # ensure that events in the session have increasing timestamps
    unstacked['timestamp'] = unstacked['timestamp'] + unstacked.groupby('user_id').cumcount()
    unstacked.rename(columns={'sequence': 'item_id'}, inplace=True)
    return unstacked

In [23]:
train_data=dataset_to_format(train_data)
test_data=dataset_to_format(test_data)
print("Train sessions: {} - Test sessions: {}".format(len(train_data), len(test_data)))

Train sessions: 9965 - Test sessions: 94895


In [111]:
groups = click_out.groupby('session_id')['timestamp','user_id'].min()
groups = groups.reset_index()
# last_session=click_out.sort_values(by=['user_id','timestamp']).groupby("user_id").last()
# impres=pd.DataFrame(last_session["impressions"])
# y_result=pd.merge(groups,impres,on=['user_id'],how='inner').set_index('session_id')
# # extract = y_result['impressions']
# # y_result.drop(labels=['impressions'], axis=1,inplace = True)
# # y_result.insert(0, 'sequence', extract)
# y_result.head()

ValueError: cannot reindex from a duplicate axis

In [63]:
train_item_based=train[train["reference"].str.isnumeric()]

In [96]:
groups = click_out.groupby('session_id')['timestamp','user_id'].min()
groups = groups.reset_index()

# group by session id and concat song_id
groups = train_item_based.groupby('session_id')

# convert item ids to string, then aggregate them to lists
aggregated = groups['reference'].agg({'sequence': lambda x: list(map(str, x))})
init_ts = groups['timestamp'].min()
users = groups['user_id'].min()

X_result = aggregated.join(init_ts).join(users)
X_result.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


KeyboardInterrupt: 

In [71]:
click_out.groupby('session_id').size()
# click_out=click_out.dropna()
# click_out.head()

Series([], dtype: int64)

In [6]:
def dataset_to_format(dataset):
    """
    Convert a list of sequences to GRU4Rec format.
    Based on this StackOverflow answer: https://stackoverflow.com/a/48532692

    :param dataset: the dataset to be transformed
    """

    lst_col = 'sequence'
    df = dataset.reset_index()
    unstacked = pd.DataFrame({
        col: np.repeat(df[col].values, df[lst_col].str.len()) for col in df.columns.drop(lst_col)}
    ).assign(**{lst_col: np.concatenate(df[lst_col].values)})[df.columns]
    # ensure that events in the session have increasing timestamps
    unstacked['timestamp'] = unstacked['timestamp'] + unstacked.groupby('user_id').cumcount()
    unstacked.rename(columns={'sequence': 'item_id'}, inplace=True)
    return unstacked

In [7]:
train_data=dataset_to_format(X_result)
test_data=dataset_to_format(y_result)
print("Train sessions: {} - Test sessions: {}".format(len(train_data), len(test_data)))

Train sessions: 3922220 - Test sessions: 3906284


In [9]:
train_data.head()

,session_id,item_id,timestamp,user_id
0,00000510f1adc,7281198,1541064087,WT30CXPIG450
1,00003f3b20954,979325,1541097696,CITFOTN2IT5P
2,000056cd97ce2,2784325,1541483846,0TBXPQCK401O
3,000056cd97ce2,3811810,1541483847,0TBXPQCK401O
4,000056cd97ce2,3811810,1541483848,0TBXPQCK401O


In [ ]:
X_reset=X_result.reset_index()
y_result=y_result.reset_index()

In [128]:
train_data.to_csv('train_data.csv',sep=',')

In [129]:
test_data.to_csv('test_data.csv',sep=',')